## Create the NCEM Object
***

In [ ]:
import numpy as np
import scanpy as sc
import pickle

import pandas as pd
import os
import anndata as ad

kwargs={'legend_loc':'on data', 'edges':False,'legend_fontsize':3,'legend_fontweight':'normal','add_outline':True,'legend_fontoutline':0.5,'wspace':0.4, 'frameon':False}
sc.settings.set_figure_params(dpi=200, frameon=False,figsize=(4, 4))
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

## Construct samplename dict
samples={}
new_output_dir="/media/jkb/Samsung2TB/Projects/SPTCR_Seq/Review_SPTCR/Final_BC_Anndat/FULL_INTEGRATION/"

samples["UKF259_SPTCR12"]=""
samples["UKF260_SPTCR13"]=""
samples["UKF334_SPTCR14"]=""
samples["UKF248_SPTCR15"]=""
samples["UKF275_SPTCR16"]=""
samples["UKF304_SPTCR17"]=""
samples["UKF296_SPTCR20"]=""
samples["UKF269_SPTCR2"]=""
samples["UKF313_SPTCR9"]=""

for key in samples.keys():
    samples[key]=f"{new_output_dir}/{key}_full_intergration.h5ad"
samples

In [5]:
ukf_name = sample_name.split("_")[0]
with open(f'../#{ukf_name}_T_ST/Cell_Subset_Anndatas/Model_w_posterior/expected_dict.pickle', 'rb') as f:
    expected_dict = pickle.load(f)

In [2]:
tcell_labels=['Cd4 Inf', 'Cd8 Cytotoxic', 'Cd8 Nk Sig','Nk','Reg T', 'Prolif T',] #'Cd4 Rest','Cd8 Em',
dc_labels=['Cdc1', 'Cdc2', 'Dc1', 'Dc2', 'Dc3','Pdc',]
bcell_labels=['B Cell', 'Plasma B',]
vasculature=['Smc', 'Perivascular Fibroblast'] #'Smc Col', 'Smc Prolif','Endo Arterial', 'Endo Capilar','Scavenging Endothelial', 'Scavenging Pericyte','Vlmc','Pericyte','Tip-Like',
brain_labels=['Astrocyte','Mast',  'Neuron',  'Oligodendrocyte', 'Opc','Rg', ]

tumor_labels=['Ac-Like', 'Mes-Like Hypoxia Independent', 'Mes-Like Hypoxia/Mhc','Npc-Like Neural', 'Npc-Like Opc',  'Opc-Like',] #'Ac-Like Prolif','Npc-Like Prolif', 'Opc-Like Prolif',
mono_labels=['Mono Anti-Infl', 'Mono Hypoxia', 'Mono Naive', ]
tam_labels=['Tam-Bdm Hypoxia/Mes', 'Tam-Bdm Mhc',  'Tam-Mg Pro-Infl I', 'Tam-Bdm Anti-Infl', 'Tam-Bdm Inf' ] #'Tam-Mg Aging Sig','Tam-Mg Pro-Infl Ii', 'Tam-Mg Prolif',
#'Stress Sig',

In [ ]:
cell_types = tcell_labels+tam_labels+tumor_labels+dc_labels+bcell_labels+vasculature

In [13]:
len(tcell_labels+tam_labels+tumor_labels+dc_labels+bcell_labels+vasculature) #+brain_labels+mono_labels

27

In [3]:
cell_types = tcell_labels+tam_labels+tumor_labels+dc_labels+bcell_labels+vasculature

## For the Cell Subsets
***

In [4]:
#ncem_output_dir="../NCEM/Niche_NCEM_Ads"
ncem_output_dir="../NCEM_Objects"

for sample_name in samples.keys():

    ukf_name = sample_name.split("_")[0]
    print(f"########## Creating the niche specific anndatas for {ukf_name} ##########")

    for niche in ["Exhausted","Reactive"]: #"Intermediate",
        print(f"############ Read the annotated TCR Anndata ############")

        adata = ad.read_h5ad(samples[sample_name])
        del adata.layers
        display(adata.obs)

        print(f"############ Read the on Expression Anndata and subset the TCR Anndata ############")
        ## use a random sc_subset anndata to do the subsetting
        path=f"../#{ukf_name}_T_ST/Cell_Subset_Anndatas/sc_subsets/Reg T_integrated_subset.h5ad"
        adata_ncem = ad.read_h5ad(path)
        adata_ncem.obs.index=adata_ncem.obs.index.str.split("_").str[3]


        print("####### Subset the Anndata by the index ########")
        adata=adata[adata.obs.index.isin(adata_ncem.obs.index.to_list()),:]
        display(adata.obs)

        adata_ncem.obs=adata.obs.copy()

        ## Replace the anndata by adata ncem
        adata=adata_ncem.copy()

        print(f"############ Add the Cell specfic expression ############")

        ########## Here You set the included cell types ##########
        cell_types = tcell_labels+tam_labels+tumor_labels+dc_labels+bcell_labels+vasculature 
        ###########################################################

        ukf_name = sample_name.split("_")[0]
        expression_dict={}

        reference_index=adata.obs.index.to_list()
        for i, n in enumerate(cell_types):
            print(i,n)
            name=n
            name = name.replace("/","_")
            path=f"../#{ukf_name}_T_ST/Cell_Subset_Anndatas/sc_subsets/{name}_integrated_subset.h5ad"

            adata_ncem=ad.read_h5ad(path)
            adata_ncem.obs.index=adata_ncem.obs.index.str.split("_").str[3]

            contained_ad=adata[adata.obs.index.isin(adata_ncem.obs.index.to_list()),:] #.obs.index.to_list()

            if len(contained_ad) < len(adata):
                print("short")
                reference_index=contained_ad.obs.index.to_list()
                adata=adata[reference_index,:]

            adata.layers[n]=adata_ncem[reference_index,:].X.copy()

        print("Cell specific Expression Ad",adata)

        print(f"############ Write the NCEM Object for {niche} ############")


        ## Subset the Anndata by the Immune Productivity
        ##############
        adata_ncem=adata[adata.obs["Immune_Productivity"] == niche].copy()

        ##############

        ########## Here You set the included cell types ##########
        cell_types = tcell_labels+tam_labels+tumor_labels+dc_labels+bcell_labels+vasculature 
        ###########################################################


        prop = adata_ncem.obs[cell_types]

        cell_expression = []
        node_types = []
        proportions = []
        spatial = []
        for i, ct in enumerate(cell_types):
            #ct = ct.replace("/","_")
            proportions.append(prop)
            cell_expression.append(adata_ncem.layers[ct].toarray())
            nt = np.zeros((prop.shape[0], len(cell_types)))
            nt[:, i] = 1
            node_types.append(nt)
            spatial.append(adata_ncem.obsm['spatial'])

        proportions = pd.DataFrame(np.concatenate(proportions), columns=cell_types)
        cell_expression = pd.DataFrame(np.concatenate(cell_expression), columns=adata_ncem.var_names)
        node_types = pd.DataFrame(np.concatenate(node_types), columns=cell_types)
        spatial = pd.DataFrame(np.concatenate(spatial))

        adata_var_names=adata_ncem.var['SYMBOL'].values.copy()


        adata = ad.AnnData(cell_expression)
        adata.obsm['proportions'] = np.array(proportions)
        adata.obsm['node_types'] = np.array(node_types)
        adata.obsm['spatial'] = np.array(spatial)

        adata.uns["node_type_names"] = {x: x for x in cell_types}


        ad.var_names = adata_var_names
        sc.pp.log1p(adata)

        sc.experimental.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True,chunksize=40000)
        h_0 = pd.DataFrame(adata.obsm['node_types'], columns=list(adata.uns['node_type_names'].values()))
        target_type = pd.DataFrame(np.array(h_0.idxmax(axis=1)), columns=["target_cell"]).reset_index()
        adata.obs = target_type
        display(adata)

        adata.write_h5ad(f"{ncem_output_dir}/#{ukf_name}_T_ST_{niche}_NCEM_Object.h5ad")

########## Creating the niche specific anndatas for UKF259 ##########
############ Read the annotated TCR Anndata ############


,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,Tumor Cells,Monocytes,TAMs,B Cells,Vasculature,CNS Cell Types,spatial_leiden_cd8,index,CD8_Cytotoxic_Composite_Score,Immune_Productivity
spot_id,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,#UKF259_T_ST,1633,7.398786,2679.0,7.893572,20.194102,28.816723,...,1.338138,0.062334,0.295481,0.064136,0.358516,1.761014,62,62,0.812218,Uninfiltrated
AAACAATCTACTAGCA-1,1,3,43,#UKF259_T_ST,3763,8.233238,8685.0,9.069468,18.618307,26.194588,...,2.889444,0.143474,0.722403,0.196728,1.118288,1.507022,5,5,-0.081323,Intermediate
AAACAGAGCGACTCCT-1,1,14,94,#UKF259_T_ST,2569,7.851661,4774.0,8.471149,18.056137,25.366569,...,2.389256,0.058263,0.449308,0.072942,0.743391,1.161195,27,27,0.104633,Uninfiltrated
AAACAGGGTCTATATT-1,1,47,13,#UKF259_T_ST,5632,8.636397,19741.0,9.890504,24.958209,32.647789,...,6.008440,0.597478,1.250880,0.588602,2.425364,1.292432,99,99,-0.034821,Intermediate
AAACAGTGTTCCTGGG-1,1,73,43,#UKF259_T_ST,1738,7.461066,2673.0,7.891331,18.481107,26.337449,...,0.890244,0.135811,0.385359,0.037339,1.011498,1.127235,28,28,-1.245459,Uninfiltrated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1,1,58,42,#UKF259_T_ST,1802,7.497207,3058.0,8.025844,19.391759,28.024853,...,2.115183,0.063611,0.311344,0.054710,0.510339,1.229673,37,37,-0.631402,Uninfiltrated
TTGTTTCATTAGTCTA-1,1,60,30,#UKF259_T_ST,2040,7.621195,3316.0,8.106816,16.887817,24.487334,...,1.533413,0.112023,0.288639,0.048629,0.925148,1.411968,32,32,-1.635941,Uninfiltrated
TTGTTTCCATACAACT-1,1,45,27,#UKF259_T_ST,2713,7.906179,6246.0,8.739857,24.095421,33.093180,...,3.279284,0.158834,0.605850,0.395232,0.778622,0.711934,63,63,1.150349,Exhausted


############ Read the on Expression Anndata and subset the TCR Anndata ############
####### Subset the Anndata by the index ########


,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,Tumor Cells,Monocytes,TAMs,B Cells,Vasculature,CNS Cell Types,spatial_leiden_cd8,index,CD8_Cytotoxic_Composite_Score,Immune_Productivity
spot_id,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,#UKF259_T_ST,1633,7.398786,2679.0,7.893572,20.194102,28.816723,...,1.338138,0.062334,0.295481,0.064136,0.358516,1.761014,62,62,0.812218,Uninfiltrated
AAACAATCTACTAGCA-1,1,3,43,#UKF259_T_ST,3763,8.233238,8685.0,9.069468,18.618307,26.194588,...,2.889444,0.143474,0.722403,0.196728,1.118288,1.507022,5,5,-0.081323,Intermediate
AAACAGAGCGACTCCT-1,1,14,94,#UKF259_T_ST,2569,7.851661,4774.0,8.471149,18.056137,25.366569,...,2.389256,0.058263,0.449308,0.072942,0.743391,1.161195,27,27,0.104633,Uninfiltrated
AAACAGGGTCTATATT-1,1,47,13,#UKF259_T_ST,5632,8.636397,19741.0,9.890504,24.958209,32.647789,...,6.008440,0.597478,1.250880,0.588602,2.425364,1.292432,99,99,-0.034821,Intermediate
AAACAGTGTTCCTGGG-1,1,73,43,#UKF259_T_ST,1738,7.461066,2673.0,7.891331,18.481107,26.337449,...,0.890244,0.135811,0.385359,0.037339,1.011498,1.127235,28,28,-1.245459,Uninfiltrated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1,1,58,42,#UKF259_T_ST,1802,7.497207,3058.0,8.025844,19.391759,28.024853,...,2.115183,0.063611,0.311344,0.054710,0.510339,1.229673,37,37,-0.631402,Uninfiltrated
TTGTTTCATTAGTCTA-1,1,60,30,#UKF259_T_ST,2040,7.621195,3316.0,8.106816,16.887817,24.487334,...,1.533413,0.112023,0.288639,0.048629,0.925148,1.411968,32,32,-1.635941,Uninfiltrated
TTGTTTCCATACAACT-1,1,45,27,#UKF259_T_ST,2713,7.906179,6246.0,8.739857,24.095421,33.093180,...,3.279284,0.158834,0.605850,0.395232,0.778622,0.711934,63,63,1.150349,Exhausted


############ Add the Cell specfic expression ############
0 Cd4 Inf
1 Cd8 Cytotoxic
2 Cd8 Nk Sig
3 Nk
4 Reg T
5 Prolif T
short
6 Tam-Bdm Hypoxia/Mes
short
7 Tam-Bdm Mhc
short
8 Tam-Mg Pro-Infl I
short
9 Tam-Bdm Anti-Infl
10 Tam-Bdm Inf
short
11 Ac-Like
12 Mes-Like Hypoxia Independent
13 Mes-Like Hypoxia/Mhc
14 Npc-Like Neural
short
15 Npc-Like Opc
short
16 Opc-Like
17 Cdc1
18 Cdc2
19 Dc1
20 Dc2
21 Dc3
short
22 Pdc
23 B Cell
24 Plasma B
25 Smc
short
26 Perivascular Fibroblast
Cell specific Expression Ad AnnData object with n_obs × n_vars = 3521 × 16166
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac', 'cell_count', 'library_id', 'mean_cell_count', '_indices', '_scvi_batch', '_scvi_labels', 'total RNA counts', 'Total cell abundance (sum_f w_sf)', 'RNA detection se

...

############ Read the on Expression Anndata and subset the TCR Anndata ############
####### Subset the Anndata by the index ########


,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,Tumor Cells,Monocytes,TAMs,B Cells,Vasculature,CNS Cell Types,spatial_leiden_cd8,index,CD8_Cytotoxic_Composite_Score,Immune_Productivity
spot_id,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,#UKF313_T_ST,2258,7.722678,5256.0,8.567316,23.706240,33.980213,...,0.521623,0.101589,0.323327,0.031227,2.316033,0.100431,90,90,0.773456,Intermediate
AAACAATCTACTAGCA-1,1,3,43,#UKF313_T_ST,2895,7.971086,7793.0,8.961109,25.202104,34.967278,...,0.792650,0.328699,0.676371,0.108749,1.970696,0.602264,20,20,-0.513679,Intermediate
AAACACCAATAACTGC-1,1,59,19,#UKF313_T_ST,312,5.746203,478.0,6.171700,37.238494,55.648536,...,0.095750,0.026923,0.056850,0.021739,0.384553,0.034832,86,86,-0.623874,Uninfiltrated
AAACAGAGCGACTCCT-1,1,14,94,#UKF313_T_ST,4348,8.377701,14868.0,9.607034,22.867904,34.241324,...,1.112325,0.278106,0.941690,0.406470,3.382629,0.353515,81,81,-0.838288,Intermediate
AAACAGCTTTCAGAAG-1,1,43,9,#UKF313_T_ST,692,6.541030,1106.0,7.009409,30.831826,42.133816,...,0.214579,0.070772,0.162842,0.020349,0.989952,0.046252,0,0,0.260611,Uninfiltrated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTGTGTGTCAAGA-1,1,31,77,#UKF313_T_ST,1772,7.480428,4354.0,8.379080,31.557189,44.097382,...,0.572822,0.095479,0.180682,0.424119,1.397926,0.277186,105,105,-0.906850,Intermediate
TTGTTTCACATCCAGG-1,1,58,42,#UKF313_T_ST,272,5.609472,404.0,6.003887,42.574257,57.425743,...,0.073195,0.027993,0.100545,0.014862,0.182672,0.053213,91,91,0.567914,Uninfiltrated
TTGTTTCATTAGTCTA-1,1,60,30,#UKF313_T_ST,342,5.837730,527.0,6.269096,37.571157,54.079696,...,0.103100,0.023068,0.074698,0.019911,0.287454,0.039787,72,72,-0.409404,Uninfiltrated


############ Add the Cell specfic expression ############
0 Cd4 Inf
short
1 Cd8 Cytotoxic
2 Cd8 Nk Sig
short
3 Nk
short
4 Reg T
5 Prolif T
short
6 Tam-Bdm Hypoxia/Mes
short
7 Tam-Bdm Mhc
short
8 Tam-Mg Pro-Infl I
short
9 Tam-Bdm Anti-Infl
short
10 Tam-Bdm Inf
short
11 Ac-Like
short
12 Mes-Like Hypoxia Independent
short
13 Mes-Like Hypoxia/Mhc
short
14 Npc-Like Neural
short
15 Npc-Like Opc
short
16 Opc-Like
short
17 Cdc1
short
18 Cdc2
short
19 Dc1
short
20 Dc2
21 Dc3
short
22 Pdc
short
23 B Cell
24 Plasma B
short
25 Smc
short
26 Perivascular Fibroblast
Cell specific Expression Ad AnnData object with n_obs × n_vars = 2733 × 16166
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac', 'cell_count', 'library_id', 'mean_cell_count', '_indices', '_scvi_batch', '_scvi_label

/home/jkb/mambaforge/envs/NCEM/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/jkb/mambaforge/envs/NCEM/lib/python3.8/site-packages/scanpy/experimental/pp/_highly_variable_genes.py:73: UserWarning: `flavor='pearson_residuals'` expects raw count data, but non-integers were found.
  warnings.warn(


AnnData object with n_obs × n_vars = 8289 × 2000
    obs: 'index', 'target_cell'
    var: 'means', 'variances', 'residual_variances', 'highly_variable_rank', 'highly_variable'
    uns: 'node_type_names', 'log1p', 'hvg'
    obsm: 'proportions', 'node_types', 'spatial'